# Stop Reinventing Pandas

The following post was presented as a talk for the [IE@DS](https://www.facebook.com/groups/173376299978861/) community, and for the [PyData meetup](https://www.meetup.com/PyData-Tel-Aviv/events/256232456/).  
All the resources for this post, including a runable notebook, can be found in the [github repo](https://github.com/DeanLa/dont_reinvent_pandas)  
blog post version here:  


<span style="font-size:2em"> [DeanLa.com](http://deanla.com/)</span>  

![slide1](slides/slide1.jpg)

![slide2](slides/slide2.jpg)

This notebook aims to show some nice ways modern Pandas makes your life easier. It is not about efficiency. I'm pretty sure using Pandas' built-in methods will be more efficient than reinventing pandas, but the main goal is to make the code easier to read, and more imoprtant - easier to write.

![slide3](slides/slide3.jpg)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use(['classic', 'ggplot', 'seaborn-poster', 'dean.style'])
%load_ext autoreload
%autoreload 2
import my_utils


# First Hacks!


Reading the data and a few housekeeping tasks. is the first place we can make our code more readable.

In [ ]:
df_io = pd.read_csv('./data.csv',index_col=0,parse_dates=['date_'])
df_io.head()

In [ ]:
df = df_io.copy().sort_values('date_').set_index('date_').drop(columns='val_updated')
df.head()

## Beautiful pipes!
One line method chaining is hard to read and prone to human error, chaining each method in its own line makes it a lot more readable.

In [ ]:
df_io\
.copy()\
.sort_values('date_')\
.set_index('date_')\
.drop(columns='val_updated')\
.head()

But it has a problem. You can't comment out and even comment in between

In [ ]:
# This block will result in an error
df_io\
.copy()\ # This is an inline comment
# This is a regular comment
.sort_values('date_')\
# .set_index('date_')\
.drop(columns='val_updated')\ 
.head()

Even an unnoticeable space character may break everything

In [ ]:
# This block will result in an error
df_io\
.copy()\
.sort_values('date_')\
.set_index('date_')\
.drop(columns='val_updated')\ 
.head()

## The Penny Drops
I like those "penny dropping" moments, when you realize you knew everything that is presented, yet it is presented in a new way you never thought of.

In [ ]:
# We can split these value inside ()
users = (134856, 195373, 295817, 294003, 262166, 121066, 129678, 307120, 258759, 277922, 220794, 192312, 318486, 314631, 306448, 297059,206892,         169046, 181703, 146200, 199876, 247904, 250884, 282989, 234280, 202520,         138064, 133577, 301053, 242157)

In [ ]:
# Penny Drop: We can also Split here
df = (df_io
        .copy() # This is an inline comment
        # This is a regular comment
        .sort_values('date_')
        .set_index('date_')
        .drop(columns='val_updated')   
)

df.head()

## Map with dict
A dict is a callable with $f(key) = value$, there for you can call `.map` with it. In this example I want to make int key codes into letter.

In [ ]:
df.event_type.map(lambda x: x+3).head()

In [ ]:
# A dict is also a callable
df['event_type'] = df.event_type.map({
    1:'A',
    5:'B',
    7:'C'
})
df.head()

# Time Series


![slide3](slides/meme-headache.jpeg)

## Resample
Task: How many events happen each hour?

### The Old Way

In [ ]:
bad = df.copy()
bad['day'] = bad.index.date
bad['hour'] = bad.index.hour
(bad
.groupby(['day','hour'])
.count()
)

* Many lines of code
* unneeded columns
* Index is not a time anymore
* **missing rows** (Did you notice?)

### A Better Way

In [ ]:
df.resample('H').count() # H is for Hour

But it's even better on non-round intervals

In [ ]:
rs = df.resample('10T').count()
# T is for Minute, and pandas understands 10 T, it will also under stand 11T if you wonder
rs.head()

[Complete list of Pandas' time abbrevations](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Period.strftime.html)

## Slice Easily
Pandas will automatically make string into timestamps, and it will understand what you want it to do.

In [ ]:
# Take only timestamp in the hour of 21:00.
rs.loc['2018-10-09 21',:]

In [ ]:
# Take all time stamps before 18:31
rs.loc[:'2018-10-09 18:31',:]

## Time Windows: Rolling, Expanding, EWM
If your Dataframe is indexed on a time index (Which 

In [ ]:
plt.style.use(['classic', 'ggplot', 'seaborn-poster', 'dean.style'])
fig, ax = plt.subplots()
rs.plot(ax=ax,linestyle='--')
(rs
 .rolling(6)
 .mean()
 .rename(columns = {'event_type':'rolling mean'})
 .plot(ax=ax)
)

rs.expanding(6).mean().rename(columns = {'event_type':'expanding mean'}).plot(ax=ax)
rs.ewm(6).mean().rename(columns = {'event_type':'ewm mean'}).plot(ax=ax)
plt.show()

### With Apply
Intuitively, windows are like GroupBy, so you can apply anything you want after the grouping, e.g.: geometric mean.

In [ ]:
fig, ax = plt.subplots()
rs.plot(ax=ax,linestyle='--')
(rs
 .rolling(6)
 .apply(lambda x: np.power(np.product(x),1/len(x)),raw=True)
 .rename(columns = {'event_type':'Rolling Geometric Mean'})
 .plot(ax=ax)
)
plt.show()

## Combine with GroupBy 🤯
Pandas has no problem with groupby and resample together. It's as simple as `groupby[col1,col2]`. In our specific case, we want to cound events in an interval per event type.

In [ ]:
per_event = (df
             .groupby('event_type')
             .resample('15T')
             .apply('count')
             .rename(columns={'event_type':'amount'})
            )
per_event.groupby('event_type').head(2)

![meme1](slides/meme-scientist.jpeg)

# Sorting

## By Values

In [ ]:
per_event.sort_values(by=['amount']).head(10)

## By Index 

In [ ]:
per_event.sort_index().head(7)

In [ ]:
per_event.sort_index(level=1).head(7)

## By Both <span style="color:red">(New in 0.23)</span>

In [ ]:
per_event.sort_values(['amount','event_type'], ascending=(False, True)).head(10)

# Stack, Unstack

## Unstack 
In this case, working with a wide format indexed on intervals, with event types as columns, will make a lot more sense.

### The Old way
Pivot table in modern pandas is more robust than it used to be. Still, it requires you to specify everything.

In [ ]:
pt = pd.pivot_table(per_event,values = 'amount',columns='event_type',index='date_')
pt.head()

### A better way
When you have just one column of values, unstack does the same easily

In [ ]:
pt = per_event.unstack('event_type')
pt.columns = pt.columns.droplevel() # Unstack creates a multiindex on columns
pt.head()

## Unstack
And some extra tricks

In [ ]:
pt.stack().head()

This looks kind of what we had expected but:
* It's a series, not a DataFrame
* The levels of the index are reversed
* The main sort is on the date, yet it used to be on the event type


### Some More Hacks



In [ ]:
stack_back = (pt
              .stack()
              .to_frame('amount') # Turn Series to DF without calling the DF constructor
              .swaplevel() # Swaps the levels of the index
              .sort_index()
             )
stack_back.head()

In [ ]:
stack_back.equals(per_event)

# Clip
Let's say, we know from domain knowledge the that an event takes place a minimum of 5 and maximum of 12 at each timestamp. We would like to fix that. In a real world example, we many time want to turn negative numbers to zeroes or some truly big numbers to sum known max.

## The Old Way
Iterate over columns and change values that meet condition.

In [ ]:
cl = pt.copy()
lb = 5
ub = 12
# Needed A loop of 3 lines
for col in ['A','B','C']:
    cl['clipped_{}'.format(col)] = cl[col]
    cl.loc[cl[col] < lb,'clipped_{}'.format(col)] = lb
    cl.loc[cl[col] > ub,'clipped_{}'.format(col)] = ub
my_utils.plot_clipped(cl) # my_utils can be found in the github repo

## A better way
`.clip(lb,ub)`

In [ ]:
cl = pt.copy()
# Beutiful One Liner
cl[['clipped_A','clipped_B','clipped_C']] = cl.clip(5,12)
my_utils.plot_clipped(cl) # my_utils can be found in the github repo

# Reindex
Now I have 3 event types from 17:00 to 23:00. Let's imagine, I know that actually I have 5 event types. I also know that the period was from 16:00 to 00:00.

In [ ]:
etypes = list('ABCYZ') # New columns
# Define a date range - Pandas will automatically make this into an index
idx = pd.date_range(start='2018-10-09 16:00:00',end='2018-10-09 23:59:00',freq=pt.index.freq)
type(idx)

In [ ]:
pt.reindex(index=idx, columns=etypes, fill_value=0).head(8)

In [ ]:
### Let's put this in a function - This will help us later.
def get_all_types_and_timestamps(df, min_date='2018-10-09 16:00:00',
                                 max_date='2018-10-09 23:59:00', etypes=list('ABCYZ')):
    ret = df.copy()
    time_idx = pd.date_range(start=min_date,end=max_date,freq='15T')
    # Indices work like set. This is a good practive so we don't override our intended index
    idx = ret.index.union(time_idx)
    etypes = df.columns.union(set(etypes))
    ret = ret.reindex(idx, columns=etypes, fill_value=0)
    return ret

# Method Chaining

## Assign
Assign is for creating new columns on the dataframes. This is instead of
`df[new_col] = function(df[old_col])`. They are both one lines, but `.assign` doesn't break the flow.

In [ ]:
pt.assign(mean_all = pt.mean(axis=1)).head()

## Pipe
Think R's `%>%` (Or rather, avoid thinking about R), `.pipe` is a method that accepts a function. `pipe`, by default, assumes the first argument of this function is a dataframe and passes the current dataframe down the pipeline. The function should return a dataframe also, if you want to continue with the chaining. Yet, it can also return any other value if you put it in the last step.  
This is incredibly valueable because it takes you one step further from "sql" where you do things "in reverse".  
$f(g(h(df)))$ = `df.pipe(h).pipe(g).pipe(f)`

In [ ]:
def add_to_col(df, col='A', n = 200):
    ret = df.copy()
    # A dataframe is mutable, if you don't copy it first, this is prone to many errors.
    # I always copy when I enter a function, even if I'm sure it shouldn't change anything.
    ret[col] = ret[col] + n
    return ret

In [ ]:
add_to_col(add_to_col(add_to_col(pt), 'B', 100), 'C',500).head()

In [ ]:
(pt
  .pipe(add_to_col)
  .pipe(add_to_col, col='B', n=100)
  .pipe(add_to_col, col='C', n=500)  
  .head(5))

You can always do this with multiple lines of `df = do_something(df)` but I think this method is more elegant.

# Beautiful Code Tells a Story
Your code is not just about making the computer do things. It's about telling a story of what you wish to happen. Sometimes other people will want to read you code. Most time, it is you 3 monhts in the future who will want to read it. Some say good code documents itself. I'm not that extreme, yet storytelling with code may save you from many lines of unnecessary comments.
The next and final block tells the story in one block. It's elegant, it tells a story. If you build utility functions and `pipe` them while following meaningful naming, they help tell a story. if you `assign` columns with meaningful names, they tell a story. you `drop`, you `apply`, you `read`, you `groupby` and you `resample` - they all tell a story.

(Well... Maybe they could have gone with better naming for `resample`)

In [ ]:
df = (pd
      .read_csv    ('./data.csv', index_col=0, parse_dates=['date_'])
      .assign      (event_type=lambda df: df.event_type.map({1: 'A', 5: 'B', 7: 'C'}))
      .sort_values ('date_')
      .set_index   ('date_')
      .drop        (columns='val_updated')
      .groupby     ('event_type')
      .resample    ('15T')
      .apply       ('count')
      .rename      (columns={'event_type': 'amount'})
      .unstack     ('event_type')
      .pipe        (my_utils.remove_multi_index)
      .pipe        (get_all_types_and_timestamps) # Remember this from before?
      .assign      (mean_event=lambda x: x.mean(axis=1))
      .loc         [:, ['mean_event']]
      .pipe        (my_utils.make_sliding_time_windows, steps_back=6)
      .dropna      ()
      )

In [ ]:
df.head()

# Bonus! 
Cool methods I've found but did not fit in the talk's flow.  


<span style="font-size:2em"> [No Time?](#You-don't-have-to-memorize-this)</span>  


In [ ]:
src = df.copy().loc[:,['mean_event']]

## Percent Change 

In [ ]:
src.assign(pct = src.pct_change()).head(11)

## Interval Index
Helps creating a "common language" when talking about time series aggregations.

In [ ]:
src = df.copy()
ir = pd.interval_range(start=df.index.min(),
                       end=df.index.max() + df.index.freq,
                       freq=df.index.freq)
type(ir)

In [ ]:
src.index = ir # Interval Index
src.loc['2018-10-09 18:37',:]


In [ ]:
df.loc['2018-10-09 18:37',:] # Datetime Index
# Will result error

## Split Strings

In [ ]:
txt = pd.DataFrame({'text':['hello','dean langsam','diving into pandas is better than reinventing it']})

In [ ]:
txt.text.str.split()

In [ ]:
txt.text.str.split(expand = True)

## Toy Examples with Pandas Testing 

In [ ]:
import pandas.util.testing as tm
tm.N, tm.K = 15, 10
st = pd.util.testing.makeTimeDataFrame() * 100
st

# Research with Style!
![so fetch](https://media.giphy.com/media/G6ojXggFcXWCs/giphy.gif)

In [ ]:
stnan = st.copy()
stnan[np.random.rand(*stnan.shape) < 0.05] = np.nan

## Basic

In [ ]:
(stnan
 .style
 .highlight_null('red')
 .highlight_max(color='steelblue', axis = 0)
 .highlight_min(color ='gold', axis = 1)
)


## Gradient 

In [ ]:
st.style.background_gradient()

## Custom

In [ ]:
def custom_style(val):
    if val < -100:
        return 'background-color:red'
    elif val > 100:
        return 'background-color:green'
    elif abs(val) <20:
        return 'background-color:yellow'
    else:
        return ''
st.style.applymap(custom_style)

## Bars

In [ ]:
(st.style
 .bar(subset=['A','D'],color='steelblue')
 .bar(subset=['J'],color=['indianred','limegreen'], align='mid')
)

# You don't have to memorize this
Just put this in the back of your mind and remember that modern Pandas has so many superpowers. Just remember they exist, and google them when you actually need them.
Always, when I feel I'm insecure about Pandas, I go back to [Greg Reda](https://twitter.com/gjreda)'s [tweet](https://twitter.com/gjreda/status/1049694953687924737):

![greg](./slides/tweet.jpg)

# Resources 
* [Modern Pandas](https://tomaugspurger.github.io/modern-1-intro.html) by Tom Augspurger
* [Basic Time Series Manipulation with Pandas](https://towardsdatascience.com/basic-time-series-manipulation-with-pandas-4432afee64ea) by Laura Fedoruk
* [Pandas Docs](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.clip.html). You don't have to thoroughly go over everything, just randomly open a page in the docs and you're sure to learn a new thing. 